# JWST Photometry: Example Usage based on TASKS.md
This notebook follows the repository’s TASKS.md to demonstrate example usage with the provided sample images. It builds a minimal config, runs the orchestrator in dry-run, and optionally runs the full pipeline.

In [ ]:
# Section 1: Load Dependencies and Configure Paths
import os, sys, json, time, io, subprocess, random
from pathlib import Path
from typing import List, Dict, Any
import numpy as np
random.seed(42); np.random.seed(42)
repo_root = Path.cwd()
tasks_path = repo_root / 'TASKS.md'
images_dir = repo_root / 'sample_images'
nb_outputs = repo_root / 'output' / 'notebook_examples'
nb_outputs.mkdir(parents=True, exist_ok=True)
sys.path.append(str(repo_root / 'src'))
print('Repo:', repo_root)
print('TASKS.md:', tasks_path.exists())
print('Images dir:', images_dir)
print('Notebook outputs:', nb_outputs)

In [ ]:
# Section 2: Validate sample_images and TASKS.md
assert tasks_path.exists(), 'TASKS.md not found'
assert images_dir.exists(), 'sample_images not found'
img_exts = ['.fits', '.fits.gz', '.fit', '.fz']
images = sorted([p for p in images_dir.iterdir() if p.suffix.lower() in img_exts or ''.join(p.suffixes).lower().endswith('.fits.gz')])
assert images, 'No FITS images found in sample_images'
print('Found images:')
for p in images:
    print(' -', p.name)

In [ ]:
# Section 3: Generate minimal config.yaml (F150W, F277W, F444W)
run_dir = nb_outputs / 'tasks_run1'
run_dir.mkdir(parents=True, exist_ok=True)
def pick(name_part: str) -> Path:
    for p in images:
        if name_part.lower() in p.name.lower():
            return p
    return images[0]
cfg_path = run_dir / 'config.yaml'
cfg = f'''\nimages:\n  F150W: {pick('F150W')}\n  F277W: {pick('F277W')}\n  F444W: {pick('F444W')}\noutput:\n  output_directory: {str(run_dir)}\n  catalog_format: fits\n'''
cfg_path.write_text(cfg)
print('Wrote config:', cfg_path)

In [ ]:
# Section 4: Orchestrator Dry-Run with Checkpoints (per TASKS Phase 7)
from pipeline import Pipeline
p = Pipeline(config_path=str(cfg_path))
p.reset()
p.run(resume=True)
print('Steps completed:', p.state.steps_completed)

## Optional: Full Pipeline Execution (from TASKS Phase 1–6)
Run the enhanced pipeline over the sample images. This requires all dependencies and may take time.

In [ ]:
# Uncomment to run full pipeline if environment is set up
# cmd = f"python {repo_root/'src'/'main.py'} --config {cfg_path} --log-level INFO"
# print(cmd)
# res = subprocess.run(['/bin/bash','-lc', cmd], text=True)
# print('Return code:', res.returncode)